In [1]:
#==================================================
#                import package
#=================================================
import math
from pynq import Overlay
import numpy as np
import math
from pynq import MMIO
from pynq import Xlnk
import time
import ctypes


In [2]:
def get_mb(mmu):
    return int(mmu.cma_stats()['CMA Memory Available']//1024//1024)

def get_kb(mmu):
    return int(mmu.cma_stats()['CMA Memory Available']//1024)

def get_bufcount(mmu):
    return int(memmanager.cma_stats()['Buffer Count'])

def print_kb(mmu):
    print("Available Memory (KB): " + str(get_kb(mmu)))

def print_mb(mmu):
    print("Available Memory (MB): " + str(get_mb(mmu)))

In [3]:
# Maximum Length: 64MB=16*1024*1024 (4byte) 16777216
DATA_SIZE=16777216

# How many 32-bit data to transfer into memory
DATA_NUM=16777216
# Setting the proper
BURST_LENGTH=256

In [4]:
#IP_base_address
IP_BASE_ADDRESS = 0x43C00000
ADDRESS_RANGE   = 0x180

#AP_CTRL Signal
XTOP_FUNCTION_CTRL_BUS_ADDR_AP_CTRL           =0x00
# Global Interrupt ENABLE
XTOP_FUNCTION_CTRL_BUS_ADDR_GIE               =0x04
# IP Interrupt Enable Register
XTOP_FUNCTION_CTRL_BUS_ADDR_IER               =0x08
# IP Interrupt Status Register
XTOP_FUNCTION_CTRL_BUS_ADDR_ISR               =0x0c
# Input Data base address
XTOP_FUNCTION_CTRL_BUS_ADDR_INPUT_R_DATA      =0x10
# Number of data to be transferred
XTOP_FUNCTION_CTRL_BUS_ADDR_DATA_NUM_DATA     =0x18
# Number of Maximum Burst
XTOP_FUNCTION_CTRL_BUS_ADDR_BURST_LENGTH_DATA =0x20



def transmit_ip(IN_Address, DATA_NUM, BURST_LENGTH):
    mmio = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE)
    
    while True:
        ap_idle =(mmio.read(XTOP_FUNCTION_CTRL_BUS_ADDR_AP_CTRL)>>2)&0X1
        if(ap_idle):
            break
    mmio.write(XTOP_FUNCTION_CTRL_BUS_ADDR_INPUT_R_DATA,IN_Address)
    mmio.write(XTOP_FUNCTION_CTRL_BUS_ADDR_DATA_NUM_DATA,DATA_NUM)
    mmio.write(XTOP_FUNCTION_CTRL_BUS_ADDR_BURST_LENGTH_DATA,BURST_LENGTH)
    start_time=time.time()
    # Start the IP
    mmio.write(XTOP_FUNCTION_CTRL_BUS_ADDR_GIE,0)
    mmio.write(XTOP_FUNCTION_CTRL_BUS_ADDR_AP_CTRL,1)
    
    while True:
        ap_done =(mmio.read(XTOP_FUNCTION_CTRL_BUS_ADDR_AP_CTRL)>>1)&0x01
        if(ap_done):
            break
    end_time=time.time()
    return (end_time-start_time)

In [5]:
overlay=Overlay("./burst_ip_150_fix.bit")
# overlay?

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [6]:
xlnk=Xlnk()
xlnk.xlnk_reset()

print('Before the allocation:',end=' ')
print_mb(xlnk)

Before the allocation: Available Memory (MB): 127


In [7]:
input_buffer=xlnk.cma_array(shape=(DATA_SIZE,), dtype=np.uint32)
INPUT_BASE = input_buffer.physical_address

input_value=np.fromfile('./64MB.bin', dtype=np.uint32)
np.copyto(input_buffer, input_value)
print("Copy Data Done!!!")

print(input_buffer[0:10])

print('After the allocation:',end=' ')
print_mb(xlnk)

Copy Data Done!!!
[3405643776 3405643777 3405643778 3405643779 3405643780 3405643781
 3405643782 3405643783 3405643784 3405643785]
After the allocation: Available Memory (MB): 49


In [8]:
print("Function Start")

"""
start_time=time.time()
transmit_ip(INPUT_BASE,DATA_NUM,BURST_LENGTH)
end_time=time.time()

execution_time=end_time-start_time
"""
execution_time=transmit_ip(INPUT_BASE,DATA_NUM,BURST_LENGTH)

print("Done! Total Transmission Time: ",str(execution_time))

Function Start
Done! Total Transmission Time:  9.1552734375e-05


# Clear the data and free the memory

In [9]:
del input_buffer
xlnk.xlnk_reset()

In [10]:
bandwidth=(64/execution_time)
print(execution_time," ",bandwidth)

9.1552734375e-05   699050.6666666666
